In [98]:
import math
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,r2_score,mean_absolute_error,mean_squared_error
from sklearn.metrics.cluster import contingency_matrix
from sklearn.tree import DecisionTreeClassifier
eps = np.finfo(float).eps
from sklearn import linear_model
from numpy import mean
from numpy import cov
from numpy import mean
from numpy import std
from numpy.linalg import eig
import pandas as pd
import copy
from sklearn.cluster import KMeans

In [82]:
df=pd.read_csv('data.csv')

In [83]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

In [84]:
train_Y=train['xAttack']
train_X=train.drop('xAttack',axis=1)
# train_X=(train_X-train_X.mean())/train_X.std()
M = mean(train_X, axis=0)
S = std(train_X,axis=0)
# print(M)
train_X=(train_X - M)/S
# train_X

In [85]:
train_Y.unique()

array(['normal', 'dos', 'probe', 'r2l', 'u2r'], dtype=object)

In [86]:
trainYnew=[]
for i in train_Y:
    if i=='normal':
        trainYnew.append(0)
    elif i=='dos':
        trainYnew.append(1)
    elif i=='r2l':
        trainYnew.append(2)
    elif i=='probe':
        trainYnew.append(3)
    elif i=='u2r':
        trainYnew.append(4)

In [87]:
mean_vec = np.mean(train_X, axis=0)
cov_mat = (train_X - mean_vec).T.dot((train_X - mean_vec)) / (train_X.shape[0]-1)

In [88]:
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

In [89]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# # Visually confirm that the list is correctly sorted by decreasing eigenvalues
# print('Eigenvalues in descending order:')
# for i in eig_pairs:
#     print(i[0])

In [90]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

In [91]:
plt.ylabel('% Variance Explained')
plt.xlabel('# of Features')
plt.title('PCA Analysis')
plt.ylim(30,100.5)
plt.style.context('seaborn-whitegrid')
plt.plot(cum_var_exp)

In [92]:
matrix_w=np.hstack((eig_pairs[0][1].reshape(29,1),eig_pairs[1][1].reshape(29,1),eig_pairs[2][1].reshape(29,1),
                    eig_pairs[3][1].reshape(29,1),eig_pairs[4][1].reshape(29,1),
                    eig_pairs[5][1].reshape(29,1),eig_pairs[6][1].reshape(29,1),
                    eig_pairs[7][1].reshape(29,1),eig_pairs[8][1].reshape(29,1),
                    eig_pairs[9][1].reshape(29,1),eig_pairs[10][1].reshape(29,1),
                    eig_pairs[11][1].reshape(29,1),eig_pairs[12][1].reshape(29,1),eig_pairs[13][1].reshape(29,1)))
Y = train_X.dot(matrix_w)

In [93]:
# Number of clusters
k = 5
# Number of training data
n = Y.shape[0]
# Number of features in the data
c = Y.shape[1]

# print c,n

# Generate random centers, here we use sigma an# data.shaped mean to ensure it represent the whole data
mean = np.mean(Y, axis = 0)

std = np.std(Y, axis = 0)
# print np.random.randn(k,c)

centers = np.random.randn(k,c)
# print centers
centers_old = np.zeros(centers.shape) # to store old centers
centers_new = deepcopy(centers) # Store new centers

clusters = np.zeros(n)
distances = np.zeros((n,k))

error = np.linalg.norm(centers_new - centers_old)
# When, after an update, the estimate of that center stays the same, exit loop
while error != 0:
    # Measure the distance to every center
    for i in range(k):
        distances[:,i] = np.linalg.norm(Y - centers[i], axis=1)
    # Assign all training data to closest center
    clusters = np.argmin(distances, axis = 1)
    
    centers_old = copy.deepcopy(centers_new)
    # Calculate mean for every cluster and update the center
    for i in range(k):
        centers_new[i] = np.mean(Y[clusters == i], axis=0)
    error = np.linalg.norm(centers_new - centers_old)

clusters

array([3, 3, 2, ..., 3, 2, 3])

In [94]:
contin=contingency_matrix(clusters,trainYnew)

In [96]:
ans=0
for i in contin:
    ans=ans+max(i)
purity=ans/float(len(train_X))
purity

0.83065868738897

In [100]:
kmeans = KMeans(n_clusters=5).fit(Y)
kmeans.labels_
contin2=contingency_matrix(kmeans.labels_,trainYnew)
ans=0
for i in contin2:
    ans=ans+max(i)
purity=ans/float(len(train_X))
purity

0.8646003910243051